<a href="https://colab.research.google.com/github/vdslab/t-taiki0620/blob/main/pke.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [28]:
!pip install git+https://github.com/boudinfl/pke.git
!pip install ginza
!pip install nltk
!pip install spacy==2.3
!python -m nltk.downloader stopwords
!pip install mojimoji

  Cloning https://github.com/boudinfl/pke.git to /tmp/pip-req-build-73kkvl5_
  Running command git clone -q https://github.com/boudinfl/pke.git /tmp/pip-req-build-73kkvl5_
  Created wheel for pke: filename=pke-1.8.1-cp37-none-any.whl size=8763774 sha256=e9bb16eb1a9477ce671ea5eccd919ed66e6417543d5ff494edeec3bcee569a6e
  Stored in directory: /tmp/pip-ephem-wheel-cache-ztys11dd/wheels/8d/24/54/6582e854e9e32dd6c632af6762b3a5d2f6b181c2992e165462
Successfully built pke
  Using cached https://files.pythonhosted.org/packages/44/87/c70718b897a9507cf66ec73bbf0b82e049e4e9d35eee5cc3a49bc32a7a90/spacy-2.3.7-cp37-cp37m-manylinux_2_17_x86_64.manylinux2014_x86_64.whl
  Found existing installation: spacy 2.3.0
    Uninstalling spacy-2.3.0:
      Successfully uninstalled spacy-2.3.0


  Using cached https://files.pythonhosted.org/packages/37/53/336f849003c88a868275959c1195c26f1bbbae777e8544b5e9fe3dd35b90/spacy-2.3.0-cp37-cp37m-manylinux1_x86_64.whl
ERROR: ja-ginza 4.0.0 has requirement spacy>=2.3.2, but you'll have spacy 2.3.0 which is incompatible.
ERROR: ginza 4.0.6 has requirement spacy<3.0.0,>=2.3.2, but you'll have spacy 2.3.0 which is incompatible.
  Found existing installation: spacy 2.3.7
    Uninstalling spacy-2.3.7:
      Successfully uninstalled spacy-2.3.7


/usr/lib/python3.7/runpy.py:125: RuntimeWarning: 'nltk.downloader' found in sys.modules after import of package 'nltk', but prior to execution of 'nltk.downloader'; this may result in unpredictable behaviour
  warn(RuntimeWarning(msg))
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
     |████████████████████████████████| 133kB 8.3MB/s 


In [1]:
import pke
import ginza
import nltk
import spacy
import pandas as pd
pke.base.lang_stopwords['ja_ginza'] = 'japanese'

In [2]:
def get_key_phrase(spacy_model, text, n = 1):
    """キーフレーズ処理

    Args:
        text ([type]): キーフレーズ対象の文字列
        n (int, optional): [description]. Defaults to 10.
        gc_collect (bool, optional): [description]. Defaults to False.

    Returns:
        [type]: [description]
    """
    #MultipartiteRankなどのキーフレーズの手法については以下のサイトが参考になります
    #https://www.ogis-ri.co.jp/otc/hiroba/technical/similar-document-search/part5.html
    extractor = pke.unsupervised.MultipartiteRank()
    extractor.load_document(input=text, language='ja_ginza', normalization=None, spacy_model= spacy_model )
    extractor.candidate_selection( pos={'NOUN', 'PROPN', 'ADJ', 'NUM'})
    extractor.candidate_weighting(threshold=0.74, method='average', alpha=1.1)
    key_phrase = extractor.get_n_best(n)
    
    return  key_phrase
    
    
#spacyに設定されているstopwordを使う
from  spacy.lang.ja import stop_words

spacy_model = spacy.load("ja_ginza")  # GiNZAモデルの読み込み
print(spacy_model)
stopwords = list(stop_words.STOP_WORDS)
nltk.corpus.stopwords.words_org = nltk.corpus.stopwords.words
nltk.corpus.stopwords.words = lambda lang : stopwords if lang == 'japanese' else nltk.corpus.stopwords.words_org(lang)

In [3]:
get_key_phrase(spacy_model,"ドラゴンボールは鳥山明の作品です。だから、今でも人気があるジャンプ作品です")

[('ドラゴン ボール', 0.5)]

In [21]:
get_key_phrase(spacy_model,"ドラゴンクエストは、誰の作品ですか？スクエニから発売されたゲームソフトです。")

[('スクエニ', 0.3962987032710885),
 ('ゲームソフト', 0.3370938981808533),
 ('ドラゴン クエスト', 0.2666073985480575)]

In [22]:
# 引用元：「東京ディズニーランド」『フリー百科事典　ウィキペディア日本語版』。
# 最終更新 2019年9月29日 (日) 04:02 UTC、URL: https://ja.wikipedia.org
text = "東京ディズニーランド、英称：Tokyo Disneyland、略称：TDL）は、" +\
"千葉県浦安市舞浜にあるディズニーリゾートを形成する日本のディズニーパーク。" +\
"年間来場者数は日本最大の約1,600万人で、世界のテーマパーク・アミューズメントパークの中でも、" +\
"フロリダ州のウォルト・ディズニー・ワールド・リゾートのマジック・キングダム、カリフォルニア州の" +\
"ディズニーランド・リゾートのディズニーランド・パークに次いで世界3位の規模を誇る[1]。オリエンタルランド" +\
"がザ・ウォルト・ディズニー・カンパニーとのライセンス契約のもと運営している[3]。"

In [4]:
import re
import mojimoji
def normalize_text(text):
  # 全部全角に変換
  result = mojimoji.han_to_zen(text)

  # 全角から半角に変換（カナは除く）
  result = mojimoji.zen_to_han(result, kana=False)

  # 全ての文字を小文字に変換
  result = result.lower()
  
  return re.sub(' ','', result)
  # return re.sub(' ', '', mojimoji.zen_to_han(text)).upper().split()

In [5]:
path = '/content/drive/MyDrive/卒業演習/tsne_+_clusters_list.json'
with open(path, encoding='utf8')as f:
  df = pd.read_json(f)

df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 22115 entries, 0 to 22114
Data columns (total 40 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   ID                   22115 non-null  int64  
 1   プロジェクトID             22115 non-null  int64  
 2   公開年度                 22115 non-null  int64  
 3   事業番号1                22115 non-null  object 
 4   事業番号2                22115 non-null  int64  
 5   事業番号3                150 non-null    float64
 6   主要政策・施策              22115 non-null  object 
 7   府省庁                  22115 non-null  object 
 8   事業名                  22115 non-null  object 
 9   事業の目的                22115 non-null  object 
 10  事業概要                 22115 non-null  object 
 11  執行額                  22115 non-null  float64
 12  objective_x          22115 non-null  float64
 13  objective_y          22115 non-null  float64
 14  cluster_in_ministry  22115 non-null  int64  
 15  ALL                  22115 non-null 

In [6]:
df = df.drop(columns=['cluster_in_ministry', 
                      'ALL', 
                      '高齢社会対策', 
                      '男女共同参画',
                      '少子化社会対策',
                      '子ども・若者育成支援',
                      '国土強靱化施策',
                      '障害者施策',
                      '地方創生',
                      '科学技術・イノベーション',
                      'ＩＴ戦略',
                      '一億総活躍推進',
                      '海洋政策',
                      '観光立国',
                      '宇宙開発利用',
                      '地球温暖化対策',
                      'ＯＤＡ',
                      '自殺対策',
                      '医療分野の研究開発関連',
                      '沖縄振興',
                      '知的財産',
                      '2020年東京オリパラ',
                      '犯罪被害者等施策',
                      'クールジャパン',
                      '食育推進',
                      'その他'])

df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 22115 entries, 0 to 22114
Data columns (total 14 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   ID           22115 non-null  int64  
 1   プロジェクトID     22115 non-null  int64  
 2   公開年度         22115 non-null  int64  
 3   事業番号1        22115 non-null  object 
 4   事業番号2        22115 non-null  int64  
 5   事業番号3        150 non-null    float64
 6   主要政策・施策      22115 non-null  object 
 7   府省庁          22115 non-null  object 
 8   事業名          22115 non-null  object 
 9   事業の目的        22115 non-null  object 
 10  事業概要         22115 non-null  object 
 11  執行額          22115 non-null  float64
 12  objective_x  22115 non-null  float64
 13  objective_y  22115 non-null  float64
dtypes: float64(4), int64(4), object(6)
memory usage: 2.4+ MB


In [7]:
df_2018 = df.iloc[0:0]
cnt = 0
for i in range(len(df)):
  if df.loc[i]['公開年度'] == 2018:
    df_2018.loc[cnt] = df.loc[i]
    cnt += 1

/usr/local/lib/python3.7/dist-packages/pandas/core/indexing.py:670: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  iloc._setitem_with_indexer(indexer, value)


In [23]:
df_2018

,ID,プロジェクトID,公開年度,事業番号1,事業番号2,事業番号3,主要政策・施策,府省庁,事業名,事業の目的,事業概要,執行額,objective_x,objective_y
0,15765,16,2018,nan,11,NaN,['その他'],内閣官房,内閣広報経費,内閣の重要施策に関する広報を首相官邸ホームページ等を通じ、国内外に対して迅速・正確に分かりや...,内閣総理大臣が国際会議等で外国を訪問した際の動向や発言を動画を用いて国民に分かりやすく伝える...,77.0,-3.439199,36.503059
1,15755,15,2018,nan,1,NaN,['その他'],内閣官房,内閣人事局経費,政府全体を通じた成果向上及び人材育成を狙いとして実施する。\n各研修に応じ、①幹部候補育成課...,１．官民合同セミナー（幹部級）・(課長級)（幹部級：本府省の幹部職員並びに民間企業の取締役ク...,23.0,57.182091,-2.059250
2,15926,52,2018,nan,124,NaN,['その他'],内閣府,人道救援物資備蓄経費,国際連合平和維持活動等に対する協力に関する法律（平成４年法律第７９号。以下「国際平和協力法」...,国際連合等から人道的な国際救援活動への物資協力の要請をうけた場合、迅速に対応できるよう人道救...,59.0,46.140083,-57.230614
3,15964,35,2018,nan,162,NaN,['クールジャパン'],内閣府,クールジャパン戦略推進経費,ゲーム・マンガ・アニメといったコンテンツ、ファッション、産品、日本食、伝統文化、デザイン等、...,クールジャパン担当大臣等の海外出張を活用したクールジャパン発信イベント、クールジャパン戦略の...,49.0,2.651179,-55.060982
4,15924,84,2018,nan,122,NaN,['その他'],内閣府,国際平和協力隊の派遣等経費,国際連合平和維持活動等に対する協力に関する法律（平成４年法律第７９号。以下「国際平和協力法」...,国際連合の要請に基づき、引き続き南スーダンにおいて、国際平和協力業務を実施する。,78.0,46.272320,-57.395348
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4368,20146,3574,2018,nan,32,NaN,['地球温暖化対策'],環境省,短期寿命気候汚染物質削減に関する国際パートナーシップ拠出金関連業務,国内におけるブラックカーボン等の短期寿命気候汚染物質（SLCP）の排出実態及び対策技術等に係...,１．短期寿命気候汚染物質削減に関する国際パートナーシップ拠出金\n CCACに対し、平成24...,327.0,-10.464338,-70.462219
4369,15941,44,2018,nan,139,NaN,['その他'],内閣府,中長期の経済運営に必要な経費,中長期の経済財政、社会基盤、社会保障等に係る重要政策の企画立案・検討に活用することを目的とし...,経済財政諮問会議等での中長期の経済財政政策の企画立案・検討に活用することを目的として、意見交...,21.6,28.345156,-26.662083
4370,18840,4895,2018,nan,204,NaN,"['観光立国', '高齢社会対策', '子ども・若者育成支援', '障害者施策', '食育推...",農林水産省,農山漁村振興交付金,都市部において農山漁村の価値が再認識されるなかで、農山漁村の活性化、自立及び維持発展を図る...,１ 地域活性化対策【補助率：定額】\n 農山漁村が持つ豊かな自然や「食」を活用した地域の活...,7886.0,-7.659595,-13.079552
4371,19154,3708,2018,nan,119,NaN,['その他'],経済産業省,中小企業・小規模事業者人材対策事業,（１）各地の中小企業・小規模事業者が必要とする人材を地域内外から発掘し、マッチング・定着など...,（１）①地域中小企業人材確保支援等事業（中核人材確保スキーム事業を除く）：各地域の中小企業・...,1454.0,-0.068404,-19.573669


In [13]:
df_2018.loc[63]['プロジェクトID']

8344

In [19]:
df_2018.loc[63]['事業概要']

'国の職員が、その職務を行うときに、故意又は過失によって違法に他人に損害を加えた際に支払う賠償、又は過大に受領した納付金等の払戻を行う。'

In [23]:
get_key_phrase(spacy_model, df_2018.loc[63]['事業概要'])

[]

In [42]:
df_pke = {}
df_enpty = []
for i in range(len(df_2018)):
  if get_key_phrase(spacy_model, df_2018.loc[i]['事業概要']) == []:
    print(i)
    df_pke[df_2018.loc[i]['プロジェクトID']] = ''
    df_enpty.append(df_2018.loc[i]['プロジェクトID'])
  else:
    df_pke[df_2018.loc[i]['プロジェクトID']] = normalize_text(get_key_phrase(spacy_model, df_2018.loc[i]['事業概要'])[0][0])

63
84
140
821
1422
1426
1461
1618
2476
2602
2835
3186
3232
3877
3964


In [43]:
print(df_2018.loc[100]['事業名'])
print(df_2018.loc[100]['事業概要'])
print(df_pke[df_2018.loc[100]['プロジェクトID']])

たばこ規制枠組条約締約国会議事務局分担金
世界保健機関（WHO）内の「たばこの規制に関する世界保健機関枠組条約」締約国会議事務局が実施する締約国会議開催の経費やたばこ規制関連ガイドラインの策定等技術的・専門的な措置等に対して拠出する。
世界保健機関


In [44]:
df_pke[8344]

''

In [46]:
import json
obj = []
for i in range(len(df_2018)):
  obj.append({'プロジェクトID':int(df_2018.loc[i]['プロジェクトID']),
             'pke':df_pke[df_2018.loc[i]['プロジェクトID']]})
  
with open('/content/drive/MyDrive/Colab Notebooks/pre-judgit/pke2018.json', 'w', encoding='utf8')as f:
  json.dump(obj, f, indent = 2, ensure_ascii=False)

In [38]:
test = {}
test['key'] = ''

In [39]:
test['key']

''